# Loading the Dataset

In [907]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats
from scipy.stats import norm, skew 


df = pd.read_csv('dataset4.csv')
df.head()


,isShowcaseListing,longitude,timeOnZillow,zestimate,daysOnZillow,zpid,taxAssessedValue,isUnmappable,priceForHDP,dateSold,state,isFeatured,isPremierBuilder,isPreforeclosureAuction,lotAreaValue,isNonOwnerOccupied,homeStatus,latitude,lotAreaUnit,bedrooms,zipcode,homeStatusForHDP,isZillowOwned,shouldHighlight,homeType,bathrooms,rentZestimate,price,city,streetAddress,country,currency,listing_sub_type,livingArea
0,False,-79.400083,6.205267e+10,NaN,718,2071870830,NaN,False,430000,1650524400000,ON,False,False,False,NaN,True,RECENTLY_SOLD,43.636951,NaN,0,M5V1B7,RECENTLY_SOLD,False,False,CONDO,1.0,NaN,430000,Toronto,600 Fleet St #409,CAN,CAD,{},NaN
1,False,-79.419914,8.036947e+10,NaN,930,2068504642,NaN,False,1452000,1632207600000,ON,False,False,False,NaN,True,RECENTLY_SOLD,43.661144,NaN,0,M6G3H2,RECENTLY_SOLD,False,False,SINGLE_FAMILY,1.0,NaN,1452000,Toronto,429 Montrose Ave,CAN,CAD,{},750.0
2,False,-79.400083,8.131987e+10,NaN,941,2071852833,NaN,False,430000,1631257200000,ON,False,False,False,NaN,True,RECENTLY_SOLD,43.636951,NaN,0,M5V1B7,RECENTLY_SOLD,False,False,CONDO,1.0,NaN,430000,Toronto,600 Fleet St UNIT 232,CAN,CAD,{},NaN
3,False,-80.344809,8.563987e+10,NaN,991,2069940188,NaN,False,1300000,1626937200000,ON,False,False,False,NaN,True,RECENTLY_SOLD,43.355007,NaN,0,N1S0A8,RECENTLY_SOLD,False,False,SINGLE_FAMILY,4.0,NaN,1300000,Cambridge,386 Salisbury Ave,CAN,CAD,{},NaN
4,False,-79.400083,6.205267e+10,NaN,718,2071870830,NaN,False,430000,1650524400000,ON,False,False,False,NaN,True,RECENTLY_SOLD,43.636951,NaN,0,M5V1B7,RECENTLY_SOLD,False,False,CONDO,1.0,NaN,430000,Toronto,600 Fleet St #409,CAN,CAD,{},NaN


In [908]:
# df['homeStatus'].value_counts()
df['homeStatusForHDP'].value_counts()

homeStatusForHDP
RECENTLY_SOLD    5042
FOR_RENT           98
FOR_SALE           80
Name: count, dtype: int64

In [909]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5220 entries, 0 to 5219
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   isShowcaseListing        5220 non-null   bool   
 1   longitude                5220 non-null   float64
 2   timeOnZillow             5100 non-null   float64
 3   zestimate                54 non-null     float64
 4   daysOnZillow             5220 non-null   int64  
 5   zpid                     5220 non-null   int64  
 6   taxAssessedValue         54 non-null     float64
 7   isUnmappable             5220 non-null   bool   
 8   priceForHDP              5220 non-null   int64  
 9   dateSold                 5220 non-null   int64  
 10  state                    5220 non-null   object 
 11  isFeatured               5220 non-null   bool   
 12  isPremierBuilder         5220 non-null   bool   
 13  isPreforeclosureAuction  5220 non-null   bool   
 14  lotAreaValue            

In [910]:
print(df.isnull().sum())

isShowcaseListing             0
longitude                     0
timeOnZillow                120
zestimate                  5166
daysOnZillow                  0
zpid                          0
taxAssessedValue           5166
isUnmappable                  0
priceForHDP                   0
dateSold                      0
state                         0
isFeatured                    0
isPremierBuilder              0
isPreforeclosureAuction       0
lotAreaValue               3230
isNonOwnerOccupied            0
homeStatus                    0
latitude                      0
lotAreaUnit                3230
bedrooms                      0
zipcode                       0
homeStatusForHDP              0
isZillowOwned                 0
shouldHighlight               0
homeType                      0
bathrooms                     0
rentZestimate              5165
price                         0
city                          0
streetAddress                 0
country                       0
currency

# Data Preprocessing and Clean up

## Removing columns
Columns like state, homeStatus, lotAreaUnit, homeType, city, streetAddress, country, and currency are categorical and should be encoded to numerical values if you plan to use them in your model.


In [911]:
columns_with_many_missing = df.columns[df.isnull().mean() > 0.5]

columns_with_low_predictive_power = [
    'zpid', 'streetAddress', 'listing_sub_type', 
    'isUnmappable', 'isShowcaseListing', 'isFeatured', 
    'isPremierBuilder', 'isPreforeclosureAuction', 
    'isNonOwnerOccupied', 'isZillowOwned', 'shouldHighlight',
    'city', 'streetAddress', 'country', 'currency', 'zpid', 'lotAreaUnit',
    'homeStatusForHDP', 'homeStatus', 'state',
     'zestimate', 'taxAssessedValue', 'rentZestimate',
     'zipcode',
    #  'dateSold',
     'priceForHDP'
]

columns_to_drop = list(set(columns_with_many_missing.to_list() + columns_with_low_predictive_power))

df_dropped = df.drop(columns=columns_to_drop)

print("columns_with_many_missing:", columns_with_many_missing)
print("Columns dropped:", columns_to_drop)
print("\nRemaining columns:", df_dropped.columns.tolist())

df_dropped.head()
df = df_dropped


columns_with_many_missing: Index(['zestimate', 'taxAssessedValue', 'lotAreaValue', 'lotAreaUnit',
       'rentZestimate', 'livingArea'],
      dtype='object')
Columns dropped: ['isPremierBuilder', 'isNonOwnerOccupied', 'isFeatured', 'country', 'homeStatus', 'isShowcaseListing', 'zpid', 'state', 'isPreforeclosureAuction', 'isZillowOwned', 'city', 'lotAreaUnit', 'isUnmappable', 'streetAddress', 'currency', 'lotAreaValue', 'rentZestimate', 'shouldHighlight', 'zestimate', 'livingArea', 'zipcode', 'priceForHDP', 'listing_sub_type', 'homeStatusForHDP', 'taxAssessedValue']

Remaining columns: ['longitude', 'timeOnZillow', 'daysOnZillow', 'dateSold', 'latitude', 'bedrooms', 'homeType', 'bathrooms', 'price']


## Categorical Data Encoding
Since homeType is a categorical variable, we'll need to encode it to use it in most machine learning models, which require numerical input.

One-Hot Encoding: This method converts the categorical variable into a series of binary columns, each representing a unique category. One-hot encoding is suitable when there is no ordinal relationship between the categories. Since homeType likely does not have a natural order, this method is appropriate.

In [912]:
df['homeType'].value_counts()

homeType
SINGLE_FAMILY    3577
CONDO             905
APARTMENT         461
TOWNHOUSE         222
MULTI_FAMILY       55
Name: count, dtype: int64

## Handling Sparse Categories
Given the distribution of your homeType categories, it appears that MANUFACTURED, MULTI_FAMILY, and TOWNHOUSE categories are relatively sparse compared to the others. To clean up the homeType column and ensure your model remains robust, you could group these less frequent categories into an "Other" category. This approach simplifies your model while retaining the information these categories provide.


In [913]:
print(df.columns)
# print(type(df))
# df['homeType'] = df['homeType'].replace(['MANUFACTURED', 'MULTI_FAMILY', 'TOWNHOUSE'], 'Other')

# print("\nNew distribution of 'homeType':")
# print(df['homeType'].value_counts())
# print(type(df))



Index(['longitude', 'timeOnZillow', 'daysOnZillow', 'dateSold', 'latitude',
       'bedrooms', 'homeType', 'bathrooms', 'price'],
      dtype='object')


In [914]:

from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Initializing the OneHotEncoder without specifying 'sparse'
encoder = OneHotEncoder(drop='first')  # Assuming 'drop' argument is supported in your version

# Fit and transform 'homeType' column
homeType_encoded_sparse = encoder.fit_transform(df[['homeType']])

# Convert the sparse matrix to a dense matrix and then to a DataFrame
homeType_encoded_df = pd.DataFrame(homeType_encoded_sparse.toarray(), columns=encoder.get_feature_names_out(['homeType']))

# Concatenate the encoded DataFrame back with the original DataFrame (excluding the original 'homeType' column)
df_final = pd.concat([df.drop('homeType', axis=1), homeType_encoded_df], axis=1)
df_final.columns
df = df_final
df.head()


,longitude,timeOnZillow,daysOnZillow,dateSold,latitude,bedrooms,bathrooms,price,homeType_CONDO,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE
0,-79.400083,6.205267e+10,718,1650524400000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0
1,-79.419914,8.036947e+10,930,1632207600000,43.661144,0,1.0,1452000,0.0,0.0,1.0,0.0
2,-79.400083,8.131987e+10,941,1631257200000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0
3,-80.344809,8.563987e+10,991,1626937200000,43.355007,0,4.0,1300000,0.0,0.0,1.0,0.0
4,-79.400083,6.205267e+10,718,1650524400000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0


## Removing duplicates

In [915]:
# Check for duplicate rows
print("Duplicates before:", df.duplicated().sum())

# Remove duplicate rows
df_removed = df.drop_duplicates()

# Check again to confirm removal
print("Duplicates after:", df_removed.duplicated().sum())

print("Number of rows:", len(df_removed))


Duplicates before: 857
Duplicates after: 0
Number of rows: 4363


## Remove where price is 0

In [916]:
# print((df['price'] == 0).sum())
# print((df['priceForHDP'] == 0).sum())
# print(((df['price'] == 0) & (df['priceForHDP'] != 0)).sum())
# df = df[df['price'] != 0]
# print('------------')
# print((df['price'] == 0).sum())
# print((df['priceForHDP'] == 0).sum())
# print(((df['price'] == 0) & (df['priceForHDP'] != 0)).sum())

# num_rows = df.shape[0]
# print(f"Number of rows: {num_rows}")


# Exploratory Data Analysis (EDA) 

## Describe

In [917]:
# display basic descriptive statistics for numerical columns
print(df.describe())


         longitude  timeOnZillow  daysOnZillow      dateSold     latitude  \
count  5220.000000  5.100000e+03   5220.000000  5.220000e+03  5220.000000   
mean    -79.417257  6.292070e+10    711.285824  1.649496e+12    43.766275   
std       0.487127  1.816487e+10    234.787378  1.798583e+10     0.301075   
min     -81.625465  2.004717e+09     -1.000000  1.617952e+12    42.755335   
25%     -79.584688  5.306723e+10    609.000000  1.634886e+12    43.650400   
50%     -79.399080  6.265758e+10    724.000000  1.649747e+12    43.708715   
75%     -79.306258  7.838234e+10    899.000000  1.659424e+12    43.794124   
max     -77.059010  9.462557e+10   1095.000000  1.710572e+12    46.104149   

          bedrooms    bathrooms         price  homeType_CONDO  \
count  5220.000000  5220.000000  5.220000e+03     5220.000000   
mean      3.677011     2.852586  1.474536e+06        0.173372   
std       1.760276     1.376055  8.993992e+05        0.378605   
min       0.000000     0.000000  1.600000e+03 

## remove nan

In [918]:
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns

print("Categorical columns:"+ str(categorical_columns))

for column in categorical_columns:
    print(f"Value counts for {column}:")
    print(df[column].value_counts())
    print("\n")

Categorical columns:Index([], dtype='object')


In [919]:

# Count columns with NaN values
nan_cols_count = df.columns[df.isna().any()].shape[0]
print(f"Number of columns with NaN values: {nan_cols_count}")

# Remove columns with NaN values
df = df.dropna(axis=1)
print(f"Number of columns with NaN values: {nan_cols_count}")


# print((df['bedrooms'] == 0).sum())
# df = df[df['bedrooms'] != 0]


Number of columns with NaN values: 1
Number of columns with NaN values: 1


# Training the Model
Implementing a Linear Regression model and split the data into training and testing sets

In [920]:
# Final columns before training
df.head()
# df.to_csv('training_dataset_saved.csv', index=False)

,longitude,daysOnZillow,dateSold,latitude,bedrooms,bathrooms,price,homeType_CONDO,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE
0,-79.400083,718,1650524400000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0
1,-79.419914,930,1632207600000,43.661144,0,1.0,1452000,0.0,0.0,1.0,0.0
2,-79.400083,941,1631257200000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0
3,-80.344809,991,1626937200000,43.355007,0,4.0,1300000,0.0,0.0,1.0,0.0
4,-79.400083,718,1650524400000,43.636951,0,1.0,430000,1.0,0.0,0.0,0.0


## Linear Regression

In [921]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

linear_model = LinearRegression()

linear_model.fit(X_train_scaled, y_train)

y_pred = linear_model.predict(X_test_scaled)



### Evaluation 

In [922]:
# Evaluating the model using Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Root Mean Squared Error (RMSE) on Test Set: {rmse}")

from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)

print(f"R-squared (R2) on Test Set: {r_squared}")

Root Mean Squared Error (RMSE) on Test Set: 675263.4630643022
R-squared (R2) on Test Set: 0.39425589772483827


## Random Forest

In [923]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming 'df' is your DataFrame and 'price' is the target variable
X = df.drop('price', axis=1)
y = df['price']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the StandardScaler (optional for Random Forest)
scaler = StandardScaler()

# Fit the scaler to the training data and transform it (optional for Random Forest)
X_train_scaled = scaler.fit_transform(X_train)

# Apply the same transformation to the test data (optional for Random Forest)
X_test_scaled = scaler.transform(X_test)

# Initialize the Random Forest Regressor
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust the number of trees

# Fit the model to the scaled training data
# Note: For Random Forest, using scaled data is not necessary, but we're keeping the scaled data for consistency
random_forest_model.fit(X_train_scaled, y_train)

# Predict using the scaled test data
y_pred = random_forest_model.predict(X_test_scaled)


### Evaluation

In [924]:
# Evaluating the model using Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Root Mean Squared Error (RMSE) on Test Set: {rmse}")

from sklearn.metrics import r2_score

r_squared = r2_score(y_test, y_pred)

print(f"R-squared (R2) on Test Set: {r_squared}")

Root Mean Squared Error (RMSE) on Test Set: 118256.10383964147
R-squared (R2) on Test Set: 0.981422390765983
